In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import ResNet50

(x_train,y_train),(x_test,y_test) = cifar10.load_data()
# 정규화
x_train = x_train / 255.0
x_test = x_test / 255.0

from tensorflow.python.framework import tensor
base_model = ResNet50(weights='imagenet', include_top=False, input_shape = x_train.shape[1:])
# 가중치 동결하지 않는 경우 기존 가중치와 함께 학습이 되어서
# 훈련데이터가 충분하지 않으면 기존 가중치의 영향에 묻혀서 제대로 반영이 안될수 있음

# 기존 모델의 가중치를 동결 해제
for layer in base_model.layers:
  layer.trainable = True

# 마지막에 우리 분류기 넣는다.
model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256,activation='relu'),
    layers.Dense(10,activation='softmax')
])
model.compile(optimizer='adam', loss = tf.keras.losses.sparse_categorical_crossentropy,metrics=['accuracy'])
model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test))

model.save('resnet50_cifar10.h5')
model = keras.models.load_model('resnet50_cifar10.h5')

!unzip '/content/test_images.zip' -d 'test_imgs'

img_files = glob('./test_imgs/*.jpg')
img_files[:3]

classs_names = ['airplane',
'automobile'							,
'bird'										,
'cat'										,
'deer'									,
'dog'										,
'frog'									,
'horse'									,
'ship'									,
'truck']
classs_names

import os
tests = []
origin_images = []
for file in img_files:
    img = Image.open(file)
    origin_images.append(img)
    x = np.array(img.resize([32,32])) / 255.0    
tests.append(x)
tests = np.array(tests)
print(tests.shape)
preds = model.predict(tests)
print(len(preds))
pred_index =  [np.argmax(pred) for pred in preds]
print(pred_index)
for i, idx in enumerate(pred_index):  # i는 예측한 이미지의 idx는 해당 클래스
    if not os.path.isdir(classs_names[idx]):
        os.mkdir(classs_names[idx])

    os.chdir(classs_names[idx])

    origin_images[i].save( img_files[i].split('/')[-1] )
    os.chdir('../')
    
    